In [1]:
from googleapiclient.discovery import build

# Reading DEVELOPER_KEY from key.txt
def readKey(filename):
    key = None
    try:
        f = open(filename, "r")
        contents = f.read()
        if (contents == ''):
            raise
        else:
            key = contents
    except:
        print("Please paste your API key in '" + filename + "' file")
        f = open(filename, "w")
    f.close()
    return key

DEVELOPER_KEY = readKey('key.txt')
YOUTUBE_API_SERVICE_NAME = 'youtube'
YOUTUBE_API_VERSION = 'v3'

In [7]:
!mongod

2019-06-25T21:11:00.020-0500 I CONTROL  [main] Automatically disabling TLS 1.0, to force-enable TLS 1.0 specify --sslDisabledProtocols 'none'
2019-06-25T21:11:00.033-0500 I CONTROL  [initandlisten] MongoDB starting : pid=88011 port=27017 dbpath=/data/db 64-bit host=Viktors-MacBook-Pro.local
2019-06-25T21:11:00.033-0500 I CONTROL  [initandlisten] db version v4.0.10
2019-06-25T21:11:00.033-0500 I CONTROL  [initandlisten] git version: c389e7f69f637f7a1ac3cc9fae843b635f20b766
2019-06-25T21:11:00.033-0500 I CONTROL  [initandlisten] allocator: system
2019-06-25T21:11:00.033-0500 I CONTROL  [initandlisten] modules: none
2019-06-25T21:11:00.033-0500 I CONTROL  [initandlisten] build environment:
2019-06-25T21:11:00.033-0500 I CONTROL  [initandlisten]     distarch: x86_64
2019-06-25T21:11:00.033-0500 I CONTROL  [initandlisten]     target_arch: x86_64
2019-06-25T21:11:00.033-0500 I CONTROL  [initandlisten] options: {}
2019-06-25T21:11:00.034-0500 I STORAGE  [initandlisten] exception in initAndLis

In [2]:
class YouTube:
    def __init__(self, KEY):
        self._YOUTUBE_API_SERVICE_NAME = 'youtube'
        self._YOUTUBE_API_VERSION = 'v3'
        self._KEY = KEY
        self.youtube = build(self._YOUTUBE_API_SERVICE_NAME, self._YOUTUBE_API_VERSION, 
                             developerKey=self._KEY)
        
        self.lastSearch = None
        self.results = []
    
    # Uses youtube.search() API method
    def search(self, q, limit=1, details=False, links=False, videoOnly=False):
        ''' 
        q - search query, (ex. "Whale Shark")
        limit - number of results (ex. 10 or 100)
        details - Retrieves more detailed information about the video (ex. True, False)
                  Cannot be True when videoOnly=False.
        links - Generate YouTube video links for each search result. 
                Can be found in video['url'] (ex. True, False) 
        videosOnly - Return only videos from the result (ex. True, False)
        '''
        if (details and not videoOnly):
            return print('You cannot use details=True and videoOnly=False together. But any other combination is valid.')
        
        searchResult = self.youtube.search().list(
            q=q,
            part='snippet',
            fields='items(id,snippet(publishedAt,title,description))',
            type='video' if videoOnly else 'video,channel,playlist',
            maxResults=limit
        ).execute()
        
        self.lastSearch = searchResult['items']
        
        # Generating YouTube video links
        if (links):
            for item in self.lastSearch:
                if (item['id']['kind'] == 'youtube#video'):
                    item['url'] = 'https://youtu.be/' + item['id']['videoId']
                elif (item['id']['kind'] == 'youtube#channel'):
                    item['url'] = 'https://www.youtube.com/channel/' + item['id']['channelId']
                    
        # Quering more details for this result
        if (details):
            for item in self.lastSearch:
                details = self.videos(item['id']['videoId'])
                for prop in details[0]:
                    try:
                        item[prop].update(details[0][prop])
                    except KeyError:
                        item[prop] = details[0][prop]
                
                
        # Appeding result to all previous search results
        self.results = self.lastSearch + self.results
        return self.lastSearch
    
    # Uses youtube.videos() API method
    def videos(self, id, limit=1, links=False):
        searchResult = self.youtube.videos().list(
            part='snippet,statistics',
            fields='items(snippet(tags),statistics)',
            id=id
        ).execute()
        
        return searchResult['items']

yt = YouTube(DEVELOPER_KEY)

In [3]:
yt.search('whale shark', limit=2, details=True, links=True, videoOnly=True)

[{'id': {'kind': 'youtube#video', 'videoId': 'E9tpPWK7sag'},
  'snippet': {'publishedAt': '2017-01-24T13:00:05.000Z',
   'title': 'Investigating the Mysterious Whale Sharks of Mafia Island | National Geographic',
   'description': "The whale sharks of Mafia Island are unusual because they don't migrate—and researchers want to know why. ➡ Subscribe: http://bit.ly/NatGeoSubscribe About ...",
   'tags': ['whale sharks',
    'Mafia Island',
    'Tanzania',
    'endangered species',
    'sharks',
    'animal migration',
    'fish',
    'Marine Megafauna Foundation',
    'research',
    'ecosystem',
    'sustained fisheries',
    'mackerel',
    'tuna',
    'conservation',
    'national geographic',
    'nat geo',
    'natgeo',
    'animals',
    'wildlife',
    'science',
    'explore',
    'discover',
    'survival',
    'nature',
    'documentary',
    'PLivjPDlt6ApRfQqtRw7JkGCLvezGeMBB2',
    'PLivjPDlt6ApRiBHpsyXWG22G8RPNZ6jlb',
    'PLivjPDlt6ApTjurXykShuUqp7LQcj9s8s']},
  'url': 'http

In [4]:
yt.search('whale shark', limit=1, details=False, links=True, videoOnly=True)

[{'id': {'kind': 'youtube#video', 'videoId': 'E9tpPWK7sag'},
  'snippet': {'publishedAt': '2017-01-24T13:00:05.000Z',
   'title': 'Investigating the Mysterious Whale Sharks of Mafia Island | National Geographic',
   'description': "The whale sharks of Mafia Island are unusual because they don't migrate—and researchers want to know why. ➡ Subscribe: http://bit.ly/NatGeoSubscribe About ..."},
  'url': 'https://youtu.be/E9tpPWK7sag'}]

In [6]:
yt.videos('8U48NfAEEGQ')

[{'snippet': {'tags': ['whaleshark',
    'shark',
    'Whale Shark (Organism Classification)',
    'Isla Mujeres (City/Town/Village)',
    'Yucatán Peninsula (Cyclone-affected Area)',
    'Mexico (Country)',
    'Caribbean Sea (Body Of Water)',
    'Coral Reef (Geographical Feature Category)',
    'Tourism (Interest)',
    'Ecotourism (Film Subject)',
    'Tourist Destination',
    'Swimming',
    'Snorkeling (Industry)',
    'Island (Geographical Feature Category)',
    'Cancún (City/Town/Village)',
    'Diving',
    'Shark (Film)',
    'Underwater']},
  'statistics': {'viewCount': '152944',
   'likeCount': '741',
   'dislikeCount': '33',
   'favoriteCount': '0',
   'commentCount': '52'}}]